In [3]:
from trending import trending_now
import pandas as pd
from googleapiclient.discovery import build

In [4]:
api_key = ""

def get_youtube(api_key):
    youtube = build("youtube", "v3", developerKey=api_key)
    return youtube

youtube = get_youtube(api_key)

In [5]:
df_videos = trending_now(['ES', 'FR', 'GB'])
list_of_ids = df_videos["video_id"].drop_duplicates().to_list()

Dataframe Created!


In [6]:
def getting_fifty_elements_lists(ids_from_videos):

    first50 = list(ids_from_videos)
    empty_list = []
    count = len(first50)

    while count > 0:
        var1 = count - 50
        var2 = count

        if var1 < 0:
            var1 = 0

        empty_list.append(first50[var1:var2])

        count -= 50

    return empty_list

list_of_fifty= getting_fifty_elements_lists(ids_from_videos=list_of_ids)

In [7]:
def stats_from_videos(videos_list, youtube):
    stats_dict = []

    for i in range(len(videos_list)):
        vid_request = youtube.videos().list(
            part=["snippet", "statistics", "contentDetails"],
            id=",".join(videos_list[i]))
        b = vid_request.execute()

        for si in range(len(b["items"])):
            bstats = b["items"][si]["statistics"]
            binfo = b["items"][si]["snippet"]
            bid = b["items"][si]["id"]
            bcontent = b["items"][si]["contentDetails"]

            stats_dict.append({
                "channel_id": binfo["channelId"],
                "video_id": bid,
                "publishedAt": binfo["publishedAt"],
                "title": binfo["title"],
                f"viewCount": bstats.get("viewCount"),
                f"likeCount": bstats.get("likeCount"),
                f"dislikeCount": bstats.get("dislikeCount"),
                f"commentCount": bstats.get("commentCount"),
                "duration": bcontent["duration"],
                "thumbnail": binfo["thumbnails"]["medium"]["url"],
                "link": f"https://youtu.be/{bid}",
                "video_lang": binfo.get("defaultAudioLanguage"),
                "categoryId": binfo["categoryId"],
                "description": binfo["description"]
            })

    vid_stats = pd.DataFrame.from_dict(stats_dict)
    channel_ids = vid_stats["channel_id"].drop_duplicates().to_list()

    return vid_stats, channel_ids

df, channel_ids  = stats_from_videos(videos_list= list_of_fifty, youtube= youtube)

In [8]:
list_of_channels = getting_fifty_elements_lists(ids_from_videos=channel_ids)

In [9]:
def data_from_channels(ch_ids, youtube):
    stats_dict = []
    upload_playlist = None

    for i in range(len(ch_ids)):
        vid_request = youtube.channels().list(
            part=["statistics", "snippet", "contentDetails"],
            id=",".join(ch_ids[i]))

        b = vid_request.execute()

        for si in range(len(b["items"])):

            bstats = b["items"][si]["statistics"]
            binfo = b["items"][si]["snippet"]
            bid = b["items"][si]["id"]

            try:
                b["items"][si]["contentDetails"]["relatedPlaylists"]["uploads"]
            except KeyError:
                upload_playlist = ""

            if upload_playlist != "":
                upload_playlist = b["items"][si]["contentDetails"]["relatedPlaylists"]["uploads"]

            stats_dict.append({
                "channel_title": binfo["title"],
                "number_of_views": bstats["viewCount"],
                "published_videos": bstats["videoCount"],
                "channel_subs": bstats.get("subscriberCount"),
                "birth_of_channel": binfo["publishedAt"],
                "country_of_the_channel": binfo.get("country"),
                "channel_id": bid,
                "channel_thumbnail": binfo["thumbnails"]["medium"]["url"],
                "upload_playlist": upload_playlist,
            })

    dataframe_output = pd.DataFrame.from_dict(
        stats_dict).drop_duplicates(subset=["channel_id"])

    upload_id = dataframe_output["upload_playlist"].tolist()

    return dataframe_output, upload_id

f_df, upload_id = data_from_channels(ch_ids=list_of_channels, youtube=youtube)

In [11]:
def get_categories_names(country, youtube):
    empty_list = []
    category_id = youtube.videoCategories().list(
        part=["snippet"],
        regionCode=country).execute()

    count = len(category_id["items"])

    for i in range(count):
        ctitle = category_id["items"][i]["snippet"]["title"]
        cid = category_id["items"][i]["id"]

        empty_list.append({
            "categoryId": cid,
            "category_title": ctitle
        })

    df = pd.DataFrame.from_dict(empty_list)

    return df

categories = get_categories_names(country="MX", youtube=youtube)

In [12]:
df.merge(f_df, how='inner', on="channel_id").merge(categories, how="inner", on="categoryId")

,channel_id,video_id,publishedAt,title,viewCount,likeCount,dislikeCount,commentCount,duration,thumbnail,...,description,channel_title,number_of_views,published_videos,channel_subs,birth_of_channel,country_of_the_channel,channel_thumbnail,upload_playlist,category_title
0,UCke6I9N4KfC968-yRcd5YRg,omPFtxcWFcM,2021-04-21T11:00:31Z,VENT MONSTER Mod in Among Us,4631609,199846,3060,8044,PT14M50S,https://i.ytimg.com/vi/omPFtxcWFcM/mqdefault.jpg,...,We add a VENT Monster Impostor in Among Us \n\...,SSundee,8335900199,2689,16700000,2009-11-28T02:52:35Z,US,https://yt3.ggpht.com/ytc/AAUvwng7wOtFA16HcyMg...,UUke6I9N4KfC968-yRcd5YRg,Gaming
1,UCocHtA1ADT6kTObipYzJoww,w4qgTREg7MQ,2021-04-21T14:26:10Z,Playing in every rank from BRONZE to PRO level...,1006363,42574,378,3991,PT15M9S,https://i.ytimg.com/vi/w4qgTREg7MQ/mqdefault.jpg,...,What it's like to play one Rocket League game ...,SunlessKhan,391905644,265,1850000,2017-01-19T05:27:25Z,US,https://yt3.ggpht.com/ytc/AAUvwnicOcdizVB2r-hk...,UUocHtA1ADT6kTObipYzJoww,Gaming
2,UCNdpD9V0PIrH1glUJ6RG4-w,pr2rAJZYvtk,2021-04-21T18:14:13Z,Luxury Underground Doomsday Mansion - FaZe Hou...,810979,34144,661,1966,PT14M39S,https://i.ytimg.com/vi/pr2rAJZYvtk/mqdefault.jpg,...,Today FaZe Clan are looking for the New FaZe H...,FaZe Clan,1092427239,1701,8630000,2010-05-31T05:21:29Z,US,https://yt3.ggpht.com/ytc/AAUvwngSDMfXCGF_UG1Y...,UUNdpD9V0PIrH1glUJ6RG4-w,Gaming
3,UC0ZV6M2THA81QT9hrVWJG3A,FTk9VD2nPuw,2021-04-19T15:00:14Z,Apex Legends | Stories from the Outlands – “No...,5790551,258036,3241,27025,PT8M18S,https://i.ytimg.com/vi/FTk9VD2nPuw/mqdefault.jpg,...,Valkyrie's been pulling smuggling jobs for yea...,Apex Legends,322617725,105,2170000,2019-01-24T18:20:16Z,US,https://yt3.ggpht.com/ytc/AAUvwnh33KmvsWBza726...,UU0ZV6M2THA81QT9hrVWJG3A,Gaming
4,UC0ZV6M2THA81QT9hrVWJG3A,oshvcDjEKZc,2021-04-22T15:00:14Z,Apex Legends – Legacy Launch Trailer,6390694,328836,2711,23951,PT3M6S,https://i.ytimg.com/vi/oshvcDjEKZc/mqdefault.jpg,...,"There’s nowhere to hide this season, Legends. ...",Apex Legends,322617725,105,2170000,2019-01-24T18:20:16Z,US,https://yt3.ggpht.com/ytc/AAUvwnh33KmvsWBza726...,UU0ZV6M2THA81QT9hrVWJG3A,Gaming
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,UCqZQXbQTF6VI_i5QWFaVxSQ,SrtemseFr4M,2021-04-25T18:30:12Z,2 MOTOCICLISTAS y UN DESTINO: BAJA CALIFORNIA ...,150805,10955,123,644,PT28M18S,https://i.ytimg.com/vi/SrtemseFr4M/mqdefault.jpg,...,"Hoy toca CAPITULAZO, por duración y por ingred...",Charly Sinewan,82757285,408,742000,2009-09-01T21:47:51Z,ES,https://yt3.ggpht.com/ytc/AAUvwnjxJ42knNCr1TWR...,UUqZQXbQTF6VI_i5QWFaVxSQ,Autos & Vehicles
567,UCzTTYntJHARp5YCxakwUepA,5L2HERKBndg,2021-04-22T19:27:22Z,"Pour une fois, je joue à autre chose qu'à MINE...",161001,19017,238,2423,PT9M42S,https://i.ytimg.com/vi/5L2HERKBndg/mqdefault.jpg,...,"Allez tester Core juste ici, en vrai c'est coo...",Fuzay²,9159073,52,335000,2015-05-13T16:08:14Z,None,https://yt3.ggpht.com/ytc/AAUvwng1tE4DPLrVXj1K...,UUzTTYntJHARp5YCxakwUepA,Travel & Events
568,UCBIv6BzHF532Bmkcqwog16A,f_Jhy2pLHFk,2021-04-25T12:50:03Z,بعد البوليساريو في اسبانيا اقوى رد على نــزع ا...,112235,6039,238,1363,PT10M47S,https://i.ytimg.com/vi/f_Jhy2pLHFk/mqdefault.jpg,...,●المرجوا الاشتراك في قناتنا الاحتياطية : http:...,صلوحي تيفي-Salouhi TV,69072543,1101,None,2011-01-10T02:52:40Z,MA,https://yt3.ggpht.com/ytc/AAUvwngdECgSq1pXWrIo...,UUBIv6BzHF532Bmkcqwog16A,Travel & Events
569,UCLYiaHiSeTC-qwjuzrKaxCg,kbSrxUXdfGY,2021-04-25T13:00:14Z,Ce piranha me sert d'appât en Amazonie - Cyril...,102003,6400,31,144,PT5M46S,https://i.ytimg.com/vi/kbSrxUXdfGY/mqdefault.jpg,...,"En Amazonie, les prédateurs ne sont pas toujou...",Cyril Chauquet,93708153,239,489000,2013-02-07T22:27:37Z,CA,https://yt3.ggpht.com/ytc/AAUvwngpLvFzpkjDmuqA...,UULYiaHiSeTC-qwjuzrKaxCg,Travel & Events
